In [3]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 24
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/24
## --- Day 24: Never Tell Me The Odds ---


It seems like something is going wrong with the snow-making process. Instead of forming snow, the water that's been absorbed into the air seems to be forming [hail](https://en.wikipedia.org/wiki/Hail) !


Maybe there's something you can do to break up the hailstones?


Due to strong, probably-magical winds, the hailstones are all flying through the air in perfectly linear trajectories. You make a note of each hailstone's **position** and **velocity** (your puzzle input). For example:


```
 19, 13, 30 @ -2,  1, -2
 18, 19, 22 @ -1, -1, -2
 20, 25, 34 @ -2, -2, -4
 12, 31, 28 @ -1, -2, -1
 20, 19, 15 @  1, -5, -3

```


Each line of text corresponds to the position and velocity of a single hailstone. The positions indicate where the hailstones are **right now** (at time0``). The velocities are constant and indicate exactly how far each hailstone will move in **one nanosecond** .


Each line of text uses the formatpx py pz @ vx vy vz``. For instance, the hailstone specified by20, 19, 15 @  1, -5, -3``has initial X position20``, Y position19``, Z position15``, X velocity1``, Y velocity-5``, and Z velocity-3``. After one nanosecond, the hailstone would be at21, 14, 12``.


Perhaps you won't have to do anything. How likely are the hailstones to collide with each other and smash into tiny ice crystals?


To estimate this, consider only the X and Y axes; **ignore the Z axis** . Looking **forward in time** , how many of the hailstones' **paths** will intersect within a test area? (The hailstones themselves don't have to collide, just test for intersections between the paths they will trace.)


In this example, look for intersections that happen with an X and Y position each at least7``and at most27``; in your actual data, you'll need to check a much larger test area. Comparing all pairs of hailstones' future paths produces the following results:


insideinside```
 Hailstone A: 19, 13, 30 @ -2, 1, -2
 Hailstone B: 18, 19, 22 @ -1, -1, -2
 Hailstones' paths will cross  the test area (at x=14.333, y=15.333).
 
 Hailstone A: 19, 13, 30 @ -2, 1, -2
 Hailstone B: 20, 25, 34 @ -2, -2, -4
 Hailstones' paths will cross  the test area (at x=11.667, y=16.667).
 
 Hailstone A: 19, 13, 30 @ -2, 1, -2
 Hailstone B: 12, 31, 28 @ -1, -2, -1
 Hailstones' paths will cross outside the test area (at x=6.2, y=19.4).
 
 Hailstone A: 19, 13, 30 @ -2, 1, -2
 Hailstone B: 20, 19, 15 @ 1, -5, -3
 Hailstones' paths crossed in the past for hailstone A.
 
 Hailstone A: 18, 19, 22 @ -1, -1, -2
 Hailstone B: 20, 25, 34 @ -2, -2, -4
 Hailstones' paths are parallel; they never intersect.
 
 Hailstone A: 18, 19, 22 @ -1, -1, -2
 Hailstone B: 12, 31, 28 @ -1, -2, -1
 Hailstones' paths will cross outside the test area (at x=-6, y=-5).
 
 Hailstone A: 18, 19, 22 @ -1, -1, -2
 Hailstone B: 20, 19, 15 @ 1, -5, -3
 Hailstones' paths crossed in the past for both hailstones.
 
 Hailstone A: 20, 25, 34 @ -2, -2, -4
 Hailstone B: 12, 31, 28 @ -1, -2, -1
 Hailstones' paths will cross outside the test area (at x=-2, y=3).
 
 Hailstone A: 20, 25, 34 @ -2, -2, -4
 Hailstone B: 20, 19, 15 @ 1, -5, -3
 Hailstones' paths crossed in the past for hailstone B.
 
 Hailstone A: 12, 31, 28 @ -1, -2, -1
 Hailstone B: 20, 19, 15 @ 1, -5, -3
 Hailstones' paths crossed in the past for both hailstones.

```


So, in this example,2``hailstones' future paths cross inside the boundaries of the test area.


However, you'll need to search a much larger test area if you want to see if any hailstones might collide. Look for intersections that happen with an X and Y position each at least200000000000000``and at most400000000000000``. Disregard the Z axis entirely.


Considering only the X and Y axes, check all pairs of hailstones' future paths for intersections. **How many of these intersections occur within the test area?** 




In [31]:
from itertools import combinations

DEBUG = False

def parse_data(data):
    return [Hailstone.from_line(line) for line in data]

class Hailstone():
    @classmethod
    def from_line(cls, line):
        line = line.replace(" ","")    
        pos,vel = line.split('@')
        pos = tuple(map(int, pos.split(",")))
        vel = tuple(map(int, vel.split(",")))
        return cls(pos, vel)

    def __init__(self, pos,vel):
        self.pos = pos
        self.vel = vel

    def as_line(self):
        return (self.pos, self.move(1).pos)

    def __repr__ (self):
        return f"({self.pos[0]},{self.pos[1]}) -> {self.vel[0]},{self.vel[1]}"
    
    def move (self, t):
        new_pos = tuple(map (lambda i, e: e+t*self.vel[i], range(len(self.pos)), self.pos ) ) 
        return Hailstone( new_pos  , self.vel )


def crossing(line1,line2):
    a,b = line1
    c,d = line2
    #https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection#Given_two_points_on_each_line
    den = (a[0]-b[0])*(c[1]-d[1]) - (a[1]-b[1])*(c[0]-d[0])

    if not den:
        return None
    
    num1 = (a[0] * b[1] - a[1] * b[0]) * (c[0]-d[0]) - (a[0] - b[0]) * (c[0] * d[1] - c[1] * d[0])
    num2 = (a[0] * b[1] - a[1] * b[0]) * (c[1]-d[1]) - (a[1] - b[1]) * (c[0] * d[1] - c[1] * d[0])

    return (num1/den, num2/den)

def is_future(hailA, hailB, cross):
    #checked and there is no vel == 0
    
    if hailA.vel[0]>0:
        if not cross[0]> hailA.pos[0]: return False
    else:
        if not cross[0] < hailA.pos[0]: return False
    
    if hailB.vel[0]>0:
        return cross[0]> hailB.pos[0]
    return cross[0] < hailB.pos[0]


def is_within_limits (value, limits):
    within =  (limits[0] <= value[0] <= limits[1]) and (limits[0] <= value[1] <= limits[1])
    return within

def solution_1(data, limits):
    hails = parse_data(data)
    if DEBUG:print("\n".join([f"{hail}" for hail in hails]) )
    count = 0
    for a,b in combinations(hails,2):
        cross = crossing( a.as_line(), b.as_line())
        if DEBUG:print(f"\nverifying {a} , {b} -> {cross}")
        if not cross:
            if DEBUG:print("parallel")
            continue
        
        elif not is_future (a,b,cross): 
            if DEBUG:print("NOT future")
            continue
        
        elif not is_within_limits( cross, limits):
            if DEBUG:print("NOT within")
            continue
        count +=1
    return count

#assert solution_1 (puzzle.test, (7,27)) == 2
#assert solution_1 (puzzle.data, (200000000000000,400000000000000)) == 11098
print( f"Solution 1:  {solution_1 (puzzle.data, (200000000000000,400000000000000))} is the number of intersections occurring within the test area") # 11098 0.3s


Solution 1:  11098 is the number of intersections occurring within the test area


In [13]:
show_problem_2(puzzle)

## --- Part Two ---


Upon further analysis, it doesn't seem like **any** hailstones will naturally collide. It's up to you to fix that!


You find a rock on the ground nearby. While it seems extremely unlikely, if you throw it just right, you should be able to **hit every hailstone in a single throw** !


You can use the probably-magical winds to reach **any integer position** you like and to propel the rock at **any integer velocity** . Now **including the Z axis** in your calculations, if you throw the rock at time0``, where do you need to be so that the rock **perfectly collides with every hailstone** ? Due toprobably-magical inertia, the rock won't slow down or change direction when it collides with a hailstone.


In the example above, you can achieve this by moving to position24, 13, 10``and throwing the rock at velocity-3, 1, 2``. If you do this, you will hit every hailstone as follows:


```
 Hailstone: 19, 13, 30 @ -2, 1, -2
 Collision time: 5
 Collision position: 9, 18, 20
 
 Hailstone: 18, 19, 22 @ -1, -1, -2
 Collision time: 3
 Collision position: 15, 16, 16
 
 Hailstone: 20, 25, 34 @ -2, -2, -4
 Collision time: 4
 Collision position: 12, 17, 18
 
 Hailstone: 12, 31, 28 @ -1, -2, -1
 Collision time: 6
 Collision position: 6, 19, 22
 
 Hailstone: 20, 19, 15 @ 1, -5, -3
 Collision time: 1
 Collision position: 21, 14, 12

```


Above, each hailstone is identified by its initial position and its velocity. Then, the time and position of that hailstone's collision with your rock are given.


After 1 nanosecond, the rock has **exactly the same position** as one of the hailstones, obliterating it into ice dust! Another hailstone is smashed to bits two nanoseconds after that. After a total of 6 nanoseconds, all of the hailstones have been destroyed.


So, at time0``, the rock needs to be at X position24``, Y position13``, and Z position10``. Adding these three coordinates together produces47``. (Don't add any coordinates from the rock's velocity.)


Determine the exact position and velocity the rock needs to have at time0``so that it perfectly collides with every hailstone. **What do you get if you add up the X, Y, and Z coordinates of that initial position?** 




In [32]:
def collinear(p0, p1, p2):
    x1, y1 = p1[0] - p0[0], p1[1] - p0[1]
    x2, y2 = p2[0] - p0[0], p2[1] - p0[1]
    return abs(x1 * y2 - x2 * y1) < 1e-12

from itertools import combinations

DEBUG = False



def solution_1(data, limits):
    hails = parse_data(data)
    if DEBUG:print("\n".join([f"{hail}" for hail in hails]) )
    count = 0
    for a,b in combinations(hails,2):
        line1 = (a.pos, a.move(1).pos)
        line2 = (b.pos, b.move(1).pos)

        cross = crossing( line1, line2)
        if DEBUG:print(f"\nverifying {a} , {b} -> {cross}")
        if not cross:
            if DEBUG:print("parallel")
            continue
        
        elif not is_future (a,b,cross): 
            if DEBUG:print("NOT future")
            continue
        
        elif not is_within_limits( cross, limits):
            if DEBUG:print("NOT within")
            continue
        count +=1
    return count

#assert solution_1 (puzzle.test, (7,27)) == 2
#assert solution_1 (puzzle.data, (200000000000000,400000000000000)) == 11098
print( f"Solution 1:  {solution_1 (puzzle.data, (200000000000000,400000000000000))} is the number of intersections occurring within the test area") # 11098 0.3s


Solution 1:  11098 is the number of intersections occurring within the test area


In [33]:
hails = parse_data(puzzle.test)

In [34]:

c1 = crossing(hails[0].as_line(), hails[1].as_line())
c2 = crossing(hails[1].as_line(), hails[2].as_line())
c3 = crossing(hails[2].as_line(), hails[3].as_line())

In [35]:
print(c1)
print(c2)
print(c2)

(14.333333333333334, 15.333333333333334)
None
None
